## Motivation
### What is your dataset?
We are working with several covid-19, mobility and transport infrastructure datasets obtained from the London datasets obtained mainly from [London Datastore](https://data.london.gov.uk/dataset) 
### Why did you choose this/these particular dataset(s)?
We decided to work with these datasets as they give the necessary context to analyse such a complex phenomena as the effect of a pandemic is.
### What was your goal for the end user's experience?
The goal of the website is to inform the user about how has Covid-19 affected the life of Londoners.

## Basic stats
The project started with the two main areas of data exploration. Mobility data from Google's London mobility dataset and bicycle hire data from the London Cycle Network, was one focus. The other area was COVID infection data from Public Health England. The mobility data was very broad, but we narrowed it down to see those areas of mobility that seemed most affected by COVID restrictions, or the absence of. The covid data had daily case numbers, but we aggregated these by week for ease of visualizing across all the boroughs.
As a part of further data exploration, we analyzed socioeconomic data to look for correlations with COVID data. These datasets included labor statistics, political affiliations, poverty and population. Geo data was also gleaned for use in making the maps of the boroughs. Not all of the plots were included in the website, but the others can be seen in this notebook.


## Data Analysis
For the majority of the project we were searching for and experimenting with different datasets that could be used to supports our general analysis into the change in mobility because of COVID-19. We first used these datasets alone and then merged the ones that we have found interesting and worthwhile to showcase in different visualisations.
The analysis breaks down the content into three main parts, introduction to the city, the socioeconomical status of the population, COVID-19 cases and restrictions and finally the change in mobility during the different periods of the lockdown. 


## Genre
Our initial plan was a slideshow genre, but we finally produced a website that incorporates different pages, almost like a partitioned poster. Visually, we used a mostly consistent visual platform, and many of the visualizations are able to be zoomed in and out for convenience.
We employed a user directed path narrative structure, with text and links to guide the user, with introductory text and commentary. Many of the graphs also employ interactivity and filtering/selection.

## Visualizations
We decided to use different libraries for our interactive visualizations. The main idea was to be able to explain certain aspects  within the restriction time period, but at the same time, offer the user the possibility to navigate by themselves. They visualization chosen allow the user to get a broad picture of how covid-19 affected London in terms of mobility patterns.


## Discussion
It was actually easier to find data than we have imagined at the start, by looking at a bunch of different datasets we had to narrow our scope. The website that was setup with hugo is really pleasant to look at and we think it does a great job at showcasing our findings in a way, that is easy for the reader to understand.
Plots that were created with folium and exported as .html ended up breaking the website we had, as a workaround we have included screenshots of these maps, not ideal, but the reader is not missing out on much.


## Contributions
Although all members contributed equally to the project, we decided to divide the work into the website content sections, ending with the following contribution table:


| Task                       | s205912 | s184361 | s202522 |
|----------------------------|---------|---------|---------|
| Motivation                 | x       |         |         |
| Genre                      |         | x       |         |
| Basic stats                |         | x       |         |
| Data analysis              |         |         | x       |
| Visualizations             | x       |         |         |
| Discussion                 |       |        | x     |
| Home section (website)     | x       |         | x       |
| Covid-19 (website)         |         | x       |         |
| Mobility section (website) | x       |         | x       |
| London section (website)   |         | x       | x       |
| Website management         | x       |         |         |

## Imports

In [301]:
!pip install pandas-profiling
!pip install geojson
!pip install shapely

import pandas as pd
import folium
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import numpy as np
import pandas_profiling
import datetime
import geojson
from shapely.geometry import shape, Point
import json
import seaborn as sns
from urllib.request import urlopen


### BOKEH
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import (
    ColumnDataSource, RangeTool, Legend, Label, LogColorMapper, ColorBar,  GeoJSONDataSource,
    HoverTool, LinearColorMapper)
from bokeh.plotting import figure
from bokeh.palettes import Viridis6, Viridis11, RdYlBu11, plasma
from bokeh.io import show, output_notebook, output_file, curdoc
from bokeh.themes import built_in_themes

### PLOTLY
import plotly.express as px


### Change from baseline 
Create mobility dataset using [Google Mobility Report](https://www.google.com/covid19/mobility/) of 2020 and 2021

In [302]:
# Upload Mobility report dataset for 2021
df = pd.read_csv('datasets/2021_GB_Region_Mobility_Report.csv')

# Upload Mobility report dataset for 2021
df0 = pd.read_csv('datasets/2020_GB_Region_Mobility_Report.csv')

# Concat datasets
df = pd.concat([df0, df])
df.drop(['country_region_code', 'country_region'], axis=1)

# Upload restrictions dataset
df1 = pd.read_csv('datasets/restrictions_daily.csv')

# Merge datatsets
data = pd.merge(df, df1, how="inner", on=["date"])

# Obtain report of the dataset
# prof = pandas_profiling.ProfileReport(data)
# prof.to_file(output_file='output.html')

data.to_csv('datasets/london_mobility_restrictions.csv')

In [303]:
# Upload Mobility report datasets

# Upload Mobility report dataset for 2021
df = pd.read_csv('datasets/2021_GB_Region_Mobility_Report.csv')

# Upload Mobility report dataset for 2021
df0 = pd.read_csv('datasets/2020_GB_Region_Mobility_Report.csv')

# Concat datasets
df = pd.concat([df0, df])


# Extract Greater London Area information
df.drop(['country_region_code', 'country_region'], axis=1)
df = df.loc[df.sub_region_1 == 'Greater London']
df.sub_region_2 = df.sub_region_2.fillna(0)
df = df.loc[df.sub_region_2 == 0]
df.date = pd.to_datetime(df.date)


# Bokeh
curdoc().theme = 'night_sky'

p = figure(plot_height=300, plot_width=800, tools=["xpan"], toolbar_location=None, title="Mobility in Greater London",
           x_axis_type="datetime", x_range=(min(df.date), max(df.date)))

line ={}
legend_items = []


for index, baseline_col in enumerate(list(df.columns.values)[-6:]):
    
    line[baseline_col] = p.line(df.date, df[baseline_col], color=Viridis6[index],
                    alpha=1, muted_color=Viridis6[index], muted_alpha=0.1, line_width=2)
    
    # Add legend and make it readable
    legend_items.append((baseline_col.replace('_', ' ')[:-28], [line[baseline_col]]))


legend = Legend(items=legend_items)
p.add_layout(legend,'left')
p.legend.click_policy = 'mute'


# Add baseline line
p.line(df.date, 0, color='black', line_width=0.5, line_dash='dashed')


p.yaxis.axis_label = 'Change from baseline'

select = figure(title="Drag the middle and edges of the selection box to change the date range above",
                plot_height=100, plot_width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None)

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2



for baseline_col in list(df.columns.values)[-6:]:
    select.line(df.date, df[baseline_col])
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
select.background_fill_alpha = 0.0
select.border_fill_alpha = 0.0
legend.background_fill_alpha = 0.0
legend.border_line_alpha = 0.0


show(column(p, select))


output_file("data/ChangeBaseline.html")


### Restrictions timeline

In [304]:
# Create dataset using the restrictions dataset
df = pd.DataFrame([
    dict(Task='schools_closed_0', Start='2020-03-23', Finish='2020-05-31', Resource='schools closed'),
    dict(Task='pubs_closed_0',  Start='2020-03-21',Finish='2020-07-02', Resource='pubs closed'),
    dict(Task='shops_closed_0',  Start='2020-03-24',Finish='2020-06-14', Resource='shops closed'),
    dict(Task='eating_places_closed_0',  Start='2020-03-21',Finish='2020-07-03', Resource='eating places closed'),
    dict(Task='schools_closed_1',  Start='2021-01-05',Finish='2021-03-07', Resource='schools closed'),
    dict(Task='pubs_closed_1',  Start='2020-11-05',Finish='2020-12-01', Resource='pubs closed'),
    dict(Task='shops_closed_1',  Start='2020-11-05',Finish='2020-12-01', Resource='shops closed'),
    dict(Task='eating_places_closed_1',  Start='2020-11-05',Finish='2020-12-01', Resource='eating places closed'),
    dict(Task='pubs_closed_2',  Start='2020-12-16',Finish='2021-04-18', Resource='pubs closed'),
    dict(Task='shops_closed_2',  Start='2020-12-20',Finish='2021-04-18', Resource='shops closed'),
    dict(Task='eating_places_closed_2',  Start='2020-12-16', Finish='2021-04-18', Resource='eating places closed'),
    dict(Task='stay_at_home_0',  Start='2020-03-24', Finish='2020-05-10', Resource='stay at home'),
    dict(Task='stay_at_home_1',  Start='2020-11-05',Finish='2020-12-02', Resource='stay at home'),
    dict(Task='stay_at_home_2',  Start='2020-12-20', Finish='2021-03-28', Resource='stay at home'),
    dict(Task='household_mixing_indoors_banned_0',  Start='2020-03-24', Finish='2020-07-03', Resource='household mixing indoors banned'),
    dict(Task='household_mixing_indoors_banned_1',  Start='2020-10-17', Finish='2021-04-18', Resource='household mixing indoors banned'),
    dict(Task='wfh_0',  Start='2020-03-17', Finish='2020-07-31', Resource='wfh'),
    dict(Task='wfh_1',  Start='2020-09-22', Finish='2021-04-18', Resource='wfh'),
    dict(Task='rule_of_6_indoors_0',  Start='2020-09-14', Finish='2020-10-16', Resource='rule of 6 indoors'),
    dict(Task='curfew_0',  Start='2020-09-24', Finish='2020-11-04', Resource='curfew'),
    dict(Task='curfew_1',  Start='2020-12-02', Finish='2020-12-15', Resource='curfew'),
    dict(Task='eat_out_to_help_out_0',  Start='2020-08-03',Finish='2020-08-30', Resource='eat out to help out'),
])

# Plot timeline of restrictions using plotly
fig = px.timeline(df, x_start="Start", x_end="Finish",
                  y="Resource", color="Resource", labels={
                     "Start": "Start",
                     "Finish": "Finish",
                     "Resource": "Restriction"
                 },
                title="Timeline of mobility restrictions", template='plotly_dark')
fig.update_xaxes(
    tickmode = 'auto',
    nticks =  20,
    tickformat="%b %y")


fig.update_layout(showlegend=False, plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)' )


fig.show()
fig.write_html("data/timeline_restrictions.html")

### Mobility infrastructure of london

In [305]:
# Obtain number of bus stops per borough, number of docking stations per borough, number of tfl stops per borough
    
with open('datasets/london_boroughs.geojson') as f:
    london = geojson.load(f)
    
with open('datasets/stops_with_lat_longs.geojson') as f:
    stops = geojson.load(f)
    
with open('datasets/cycle_hire_locations.geojson') as f:
    stations = geojson.load(f)   

with open('datasets/tfl.geojson') as f:
    tfl = geojson.load(f)   
    
        
# Transform coordenates to Points
bus_points = []
for i in range(len(stops['features'])):
    bus_points.append(Point(stops['features'][i]['geometry']['coordinates']
                  [0], stops['features'][i]['geometry']['coordinates'][1]))

docking_points = []
for i in range(len(stations['features'])):
    docking_points.append(Point(stations['features'][i]['properties']['longitude'],
                   stations['features'][i]['properties']['latitude']))
    
tfl_points = []
for i in range(len(tfl['features'])):
     tfl_points.append(Point(tfl['features'][i]['geometry']['coordinates']
                  [0], tfl['features'][i]['geometry']['coordinates'][1]))

# Check if borough contains each point        
for feature in london['features']:
    polygon = shape(feature['geometry'])
    feature['properties']['stops_counter']  = 0
    feature['properties']['docking_counter']  = 0
    feature['properties']['tfl_counter']  = 0
    
    # Count stops
    for point in bus_points:
        if polygon.contains(point):
            feature['properties']['stops_counter'] +=1
            
    # Count docking stations       
    for point in docking_points:
        if polygon.contains(point):
            feature['properties']['docking_counter'] +=1
            
    # Count docking stations       
    for point in tfl_points:
        if polygon.contains(point):
            feature['properties']['tfl_counter'] +=1
            
with open('datasets/london_stops.geojson', 'w') as f:
    json.dump(london, f)            

#### Bus

In [306]:
# Plot Bus Stops per Borough

with open('datasets/london_stops.geojson') as f:
    london_source = GeoJSONDataSource(geojson=f.read())


# Define color mapper    
color_mapper = LinearColorMapper(palette=Viridis11, low=0, high=1200)

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

curdoc().theme = 'night_sky'

p = figure(title="Number of bus stops per borough", tools=TOOLS, x_axis_location=None,
           y_axis_location=None, width=600, height=400)
p.grid.grid_line_color = None


# Plot map of Greater London
p.patches('xs', 'ys', source=london_source,
          fill_color={'field': 'stops_counter', 'transform': color_mapper},
          fill_alpha=1.0, line_color="black", line_width=0.5)

# Add legend
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0))

p.add_layout(color_bar, 'right')

p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
color_bar.background_fill_alpha = 0.0


hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Borough", "@name"), ("Number of stops", "@stops_counter")]

output_file("data/London_bus_stops.html", title="Number of bus stops per borough")

show(p)

#### Cycle

In [307]:
# Plot cycle hire docking stations per Borough

with open('datasets/london_stops.geojson') as f:
    london_source = GeoJSONDataSource(geojson=f.read())

# Define color mapper       
color_mapper = LinearColorMapper(palette=Viridis11, low=0, high=200)

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

curdoc().theme = 'night_sky'

p = figure(title="Number of Santander Cycle Hire docking stations per borough", tools=TOOLS, x_axis_location=None,
           y_axis_location=None, width=600, height=400)
p.grid.grid_line_color = None


# Plot map of Greater London
p.patches('xs', 'ys', source=london_source,
          fill_color={'field': 'docking_counter', 'transform': color_mapper},
          fill_alpha=1.0, line_color="black", line_width=0.5)


color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0))

p.add_layout(color_bar, 'right')

p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
color_bar.background_fill_alpha = 0.0

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Borough", "@name"), ("Number of docking stations", "@docking_counter")]

output_file("data/London_docking_stations.html", title="Number of bus stops per borough")

show(p)

In [308]:
# Plot number of cycle hires

cycle_hires = pd.read_excel('datasets/tfl-daily-cycle-hires.xlsx', sheet_name='Data')


cycle_hires = cycle_hires.loc[cycle_hires['Day']>'2017-01-01']

curdoc().theme = 'night_sky'

p = figure(plot_height=300, plot_width=800, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_range=(datetime.datetime(2017,1,1), datetime.datetime(2021, 4, 1)))

p.circle(cycle_hires['Day'], cycle_hires['Number of Bicycle Hires'])

p.yaxis.axis_label = 'Bicycle Hires'





select = figure(title="Drag the middle and edges of the selection box to change the date range above",
                plot_height=100, plot_width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None)



p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
select.background_fill_alpha = 0.0
select.border_fill_alpha = 0.0


range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line(cycle_hires['Day'], cycle_hires['Number of Bicycle Hires'])
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))

output_file("data/CycleHires.html")

In [309]:
# Plot cycle routes
with open('datasets/london_boroughs.geojson') as f:
    london_source = GeoJSONDataSource(geojson=f.read())

with open('datasets/CycleRoutes.geojson') as f:
    cycle_source = GeoJSONDataSource(geojson=f.read())

    
with open('datasets/cycle_hire_locations.geojson') as f:
    docking_source = GeoJSONDataSource(geojson=f.read())


TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(title="Cycle Routes", tools=TOOLS, x_axis_location=None,
           y_axis_location=None, width=600, height=400)
p.grid.grid_line_color = None


# Plot map of Greater London
p.patches('xs', 'ys', source=london_source,
          fill_color=Viridis6[0],
          fill_alpha=0.5, line_color="black", line_width=0.5)

# Plot cycle routes on top
p.multi_line('xs', 'ys', source=cycle_source, line_alpha=1, line_color=Viridis6[5],
             line_width=0.7)

p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
        
hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Borough", "@name")]

output_file("data/Cycle_Routes.html", title="Cycle Routes")

show(p)

#### TLF STATIONS (Tube, DLR, Overground, Tramlink, Emirates Air Line and Crossrail)

In [310]:
# Plot TFL stations per Borough

with open('datasets/london_stops.geojson') as f:
    london_source = GeoJSONDataSource(geojson=f.read())

# Define color mapper 
color_mapper = LinearColorMapper(palette=Viridis11, low=0, high=40)

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

curdoc().theme = 'night_sky'

p = figure(title="Number of TFL Stations per borough", tools=TOOLS, x_axis_location=None,
           y_axis_location=None, width=600, height=400)
p.grid.grid_line_color = None


# Plot map of Greater London
p.patches('xs', 'ys', source=london_source,
          fill_color={'field': 'tfl_counter', 'transform': color_mapper},
          fill_alpha=1.0, line_color="black", line_width=0.5)


color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0))

p.add_layout(color_bar, 'right')

p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
color_bar.background_fill_alpha = 0.0

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Borough", "@name"), ("Number of TFL stations", "@tfl_counter")]

output_file("data/London_tfl_stations.html", title="Number of TFL Stations per borough")

show(p)

Maps made with folium - Google Mobility Data

In [314]:
mobility = pd.read_csv('datasets/cleaned_sub_region_2_data.csv')

Create a dataframe and fill for the first wave

In [315]:
mobility_mean_first = pd.DataFrame(columns=['name', 'retail_and_recreation_percent_change_from_baseline_first', 'grocery_and_pharmacy_percent_change_from_baseline_first', 'parks_percent_change_from_baseline_first', 'transit_stations_percent_change_from_baseline_first', 'workplaces_percent_change_from_baseline_first', 'residential_percent_change_from_baseline_first'])

In [316]:
for index, borough in enumerate(mobility.sub_region_2.unique()):
    mask = ('2020-03-24' <= mobility['date']) & (mobility['date'] <= '2020-03-30' ) & (mobility['sub_region_2'] == borough)
    borough_df = mobility.loc[mask]
    mobility_mean_first.loc[index] = [
        borough, 
        borough_df.retail_and_recreation_percent_change_from_baseline.mean(), 
        borough_df.grocery_and_pharmacy_percent_change_from_baseline.mean(), 
        borough_df.parks_percent_change_from_baseline.mean(), 
        borough_df.transit_stations_percent_change_from_baseline.mean(), 
        borough_df.workplaces_percent_change_from_baseline.mean(), 
        borough_df.residential_percent_change_from_baseline.mean()
        ]

In [317]:
mobility_mean_first

,name,retail_and_recreation_percent_change_from_baseline_first,grocery_and_pharmacy_percent_change_from_baseline_first,parks_percent_change_from_baseline_first,transit_stations_percent_change_from_baseline_first,workplaces_percent_change_from_baseline_first,residential_percent_change_from_baseline_first
0,City of London,-95.142857,-80.714286,-90.571429,-91.000000,-83.200000,NaN
1,Westminster,-94.428571,-64.857143,-81.857143,-89.428571,-77.000000,35.000000
2,Barking and Dagenham,-69.000000,-30.000000,-1.571429,-60.857143,-61.571429,24.428571
3,Barnet,-75.714286,-28.142857,17.714286,-70.428571,-72.000000,29.428571
4,Bexley,-75.857143,-30.000000,-8.571429,-68.285714,-64.428571,26.000000
5,Brent,-74.857143,-30.285714,14.000000,-66.428571,-67.000000,27.571429
6,Bromley,-78.428571,-27.000000,-11.000000,-73.285714,-72.000000,28.428571
7,Camden,-89.571429,-58.571429,-42.428571,-88.000000,-78.857143,36.000000
8,Croydon,-79.857143,-27.714286,23.428571,-72.428571,-66.285714,27.285714
9,Ealing,-73.714286,-29.142857,3.000000,-70.714286,-68.000000,28.142857


Create a dataframe and fill for the second wave

In [318]:
mobility_mean_second = pd.DataFrame(columns=['name', 'retail_and_recreation_percent_change_from_baseline_second', 'grocery_and_pharmacy_percent_change_from_baseline_second', 'parks_percent_change_from_baseline_second', 'transit_stations_percent_change_from_baseline_second', 'workplaces_percent_change_from_baseline_second', 'residential_percent_change_from_baseline_second'])

In [319]:
for index, borough in enumerate(mobility.sub_region_2.unique()):
    mask = ('2020-11-05' <= mobility['date']) & (mobility['date'] <= '2020-11-11' ) & (mobility['sub_region_2'] == borough)
    borough_df = mobility.loc[mask]
    mobility_mean_second.loc[index] = [
        borough, 
        borough_df.retail_and_recreation_percent_change_from_baseline.mean(),
        borough_df.grocery_and_pharmacy_percent_change_from_baseline.mean(), 
        borough_df.parks_percent_change_from_baseline.mean(), 
        borough_df.transit_stations_percent_change_from_baseline.mean(), 
        borough_df.workplaces_percent_change_from_baseline.mean(), 
        borough_df.residential_percent_change_from_baseline.mean()]

In [320]:
mobility_mean_second

,name,retail_and_recreation_percent_change_from_baseline_second,grocery_and_pharmacy_percent_change_from_baseline_second,parks_percent_change_from_baseline_second,transit_stations_percent_change_from_baseline_second,workplaces_percent_change_from_baseline_second,residential_percent_change_from_baseline_second
0,City of London,-89.428571,-71.285714,-85.714286,-79.571429,-66.200000,NaN
1,Westminster,-86.714286,-48.142857,-57.571429,-76.857143,-57.857143,21.714286
2,Barking and Dagenham,-39.857143,-7.000000,35.000000,-38.285714,-33.285714,12.714286
3,Barnet,-50.571429,-11.571429,55.428571,-49.714286,-44.857143,18.000000
4,Bexley,-52.714286,-11.000000,39.571429,-49.714286,-40.714286,16.571429
5,Brent,-51.285714,-7.142857,67.142857,-44.857143,-42.142857,15.857143
6,Bromley,-55.857143,-9.857143,24.142857,-52.285714,-44.857143,19.285714
7,Camden,-79.714286,-38.571429,0.857143,-74.428571,-54.571429,22.142857
8,Croydon,-56.714286,-7.857143,47.142857,-53.000000,-37.857143,17.000000
9,Ealing,-45.285714,-10.714286,41.714286,-50.142857,-41.857143,17.000000


Create 2 dfs so that we can join them on 'name'

In [321]:
data_final_first = pd.DataFrame(columns=[
    'name',
    'retail_and_recreation_percent_change_from_baseline_first',
    'grocery_and_pharmacy_percent_change_from_baseline_first',
    'parks_percent_change_from_baseline_first',
    'transit_stations_percent_change_from_baseline_first',
    'workplaces_percent_change_from_baseline_first',
    'residential_percent_change_from_baseline_first'
])

In [322]:
data_final_second = pd.DataFrame(columns=[
    'name',
    'retail_and_recreation_percent_change_from_baseline_second',
    'grocery_and_pharmacy_percent_change_from_baseline_second',
    'parks_percent_change_from_baseline_second',
    'transit_stations_percent_change_from_baseline_second',
    'workplaces_percent_change_from_baseline_second',
    'residential_percent_change_from_baseline_second'
])

In [323]:
df_final = data_final_first.join(data_final_second.set_index('name'), on='name')

load geojson

In [326]:
london_boroughs = f"datasets/london_boroughs.geosjon"

From here we create the maps for each category and each wave

In [327]:
rr1 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['retail_and_recreation_percent_change_from_baseline_first'].notna()],
                  columns = ['name', 'retail_and_recreation_percent_change_from_baseline_first'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Retail and Recreation \% Change from Baseline (First Lockdown)',
                  name="Retail and Recreation \% Change from Baseline (First Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(rr1)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/london_boroughs.geosjon'

In [ ]:
gp1 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['grocery_and_pharmacy_percent_change_from_baseline_first'].notna()],
                  columns = ['name', 'grocery_and_pharmacy_percent_change_from_baseline_first'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Grocery and Pharmacy \% Change from Baseline (First Lockdown)',
                  name="Grocery and Pharmacy \% Change from Baseline (First Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(gp1)

In [ ]:
p1 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['parks_percent_change_from_baseline_first'].notna()],
                  columns = ['name', 'parks_percent_change_from_baseline_first'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Parks \% Change from Baseline (First Lockdown)',
                  name="Parks \% Change from Baseline (First Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(p1)

In [ ]:
ts1 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['transit_stations_percent_change_from_baseline_first'].notna()],
                  columns = ['name', 'transit_stations_percent_change_from_baseline_first'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Transit Stations \% Change from Baseline (First Lockdown)',
                  name="Transit Stations \% Change from Baseline (First Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(ts1)

In [ ]:
w1 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['workplaces_percent_change_from_baseline_first'].notna()],
                  columns = ['name', 'workplaces_percent_change_from_baseline_first'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Workplaces \% Change from Baseline (First Lockdown)',
                  name="Workplaces \% Change from Baseline (First Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(w1)

In [ ]:
rr2 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['retail_and_recreation_percent_change_from_baseline_second'].notna()],
                  columns = ['name', 'retail_and_recreation_percent_change_from_baseline_second'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Retail and Recreation \% Change from Baseline (Second Lockdown)',
                  name="Retail and Recreation \% Change from Baseline (Second Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(rr2)

In [ ]:
gp2 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['grocery_and_pharmacy_percent_change_from_baseline_second'].notna()],
                  columns = ['name', 'grocery_and_pharmacy_percent_change_from_baseline_second'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Grocery and Pharmacy \% Change from Baseline (Second Lockdown)',
                  name="Grocery and Pharmacy \% Change from Baseline (Second Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(gp2)

In [ ]:
p2 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['parks_percent_change_from_baseline_second'].notna()],
                  columns = ['name', 'parks_percent_change_from_baseline_second'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Parks \% Change from Baseline (Second Lockdown)',
                  name="Parks \% Change from Baseline (Second Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(p2)

In [ ]:
ts2 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['transit_stations_percent_change_from_baseline_second'].notna()],
                  columns = ['name', 'transit_stations_percent_change_from_baseline_second'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Transit Stations \% Change from Baseline (Second Lockdown)',
                  name="Transit Stations \% Change from Baseline (Second Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(ts2)

In [ ]:
w2 = folium.Map([51.5074, -0.1278], tiles='Stamen Toner', zoom_start=10)

folium.Choropleth(geo_data = london_boroughs,
                  data = df_final[df_final['workplaces_percent_change_from_baseline_second'].notna()],
                  columns = ['name', 'workplaces_percent_change_from_baseline_second'],
                  key_on = 'feature.properties.name',
                  legend_name = 'Workplaces \% Change from Baseline (Second Lockdown)',
                  name="Workplaces \% Change from Baseline (Second Lockdown)",
                  fill_color='YlGnBu',
                  overlay=False,
                  show=False).add_to(w2)

### Covid cases

In [ ]:
# data of all london covid cases
cases_london = pd.read_csv('phe_cases_age_london.csv')
cases_london['date'] = pd.to_datetime(cases_london['date'], format = '%Y-%m-%d')
cases_london = cases_london.groupby(['date']).agg({'cases': 'sum'})
cases_london.insert(0, 'area_code', '')
cases_london['area_code'] = 'E12000007'
cases_london.insert(0, 'area_name', '')
cases_london['area_name'] = 'London'

In [ ]:
# data of covid cases by borough
cases_boroughs = pd.read_csv('phe_cases_london_boroughs.csv')
cases_boroughs = cases_boroughs.replace(['Hackney and City of London'], 'Hackney')

In [ ]:
# clean dataframe and aggregate by new cases
cases_boroughs.drop(['area_code','total_cases'],axis=1, inplace=True)
cases_boroughs['date'] = pd.to_datetime(cases_boroughs['date'], format = '%Y-%m-%d')

cases_boroughs = cases_boroughs.groupby(['date','area_name']).agg({'new_cases': 'sum'}).unstack()
cases_boroughs.columns = cases_boroughs.columns.droplevel()
areas = list(cases_boroughs.columns)

In [ ]:
# plot of total cases
cases = cases_london.loc[:,'cases']

# Plot timeline of restrictions using plotly
fig = px.line(cases,
                title='Total COVID cases for London', template='plotly_dark',
                labels={'value':'Cases', 'date':'Date'})
fig.update_xaxes(
    tickmode = 'auto',
    nticks =  20,
    tickformat="%b %y")
fig.update_layout(showlegend=False, 
                  xaxis_showgrid=False, 
                  yaxis_showgrid=False,
                  plot_bgcolor = 'rgba(0, 0, 0, 0)', 
                  paper_bgcolor = 'rgba(0, 0, 0, 0)' )
fig.show()
fig.write_html('totalcases.html')

In [ ]:
# bokeh plot of cases by borough
numlines=len(areas)
colors_list = plasma(32)[0:numlines]
xs=[cases_boroughs.index.values]*numlines
ys=[cases_boroughs[name].values for name in cases_boroughs]
legend_items = []

curdoc().theme = "dark_minimal"

p = figure(title='New COVID cases for each borough', width=1200, height=800, x_axis_type="datetime")

for (colr, leg, x, y ) in zip(colors_list, areas, xs, ys):
    my_plot = p.line(x, y, color= colr, alpha=0.8)
    legend_items.append((leg, [my_plot]))

legend = Legend(items=legend_items)
p.add_layout(legend,'left')
p.legend.click_policy = 'hide'
p.legend.label_text_font_size = "10px"
p.background_fill_alpha = 0.0
p.border_fill_alpha = 0.0
legend.background_fill_alpha = 0.0
legend.border_line_alpha = 0.0 
output_file('boroughcases.html', title='New COVID cases for each borough')
show(p)

In [ ]:
# prepare dataframe of covid cases by borough
cases = pd.read_csv('phe_cases_london_boroughs.csv')
cases = cases.replace(['Hackney and City of London'], 'Hackney')
cases['date'] = pd.to_datetime(cases['date'], format = '%Y-%m-%d')
cases['week'] = cases['date'].datetime.year.astype(str) + ' week ' + cases['date'].datetime.week.astype(str)

# data for first lockdown
cases1 = cases.loc[(cases['date']>='2020-11-05') & (cases['date']<='2021-02-01')].reset_index()
cases1 = cases1.groupby(['area_name','week']).sum().reset_index()

In [ ]:
# plot data for first lockdown
fig = px.choropleth_mapbox(cases1, geojson=boroughs, locations='area_name', color='new_cases',
                           mapbox_style='stamen-toner',
                           color_continuous_scale='YlOrRd',
                           zoom=8, center = {"lat": 51.5074, "lon": -0.07},
                           animation_frame='week',
                           labels={'new_cases':'New cases', 'area_name':'Borough', 'week':'Week'},
                           title='Weekly new COVID-19 cases during Winter \'20-\'21 lockdown',
                           template='plotly_dark',
                           opacity=0.5)
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)'},margin={"r":0,"t":40,"l":0,"b":0})
fig.show()
fig.write_html("winter_dark.html")

In [ ]:
# data for second wave lockdown
cases2 = cases.loc[(cases['date']>='2020-03-21') & (cases['date']<='2020-06-14')].reset_index()
cases2 = cases2.groupby(['area_name','week']).sum().reset_index()

In [ ]:
# plot data for second wave lockdown
fig = px.choropleth_mapbox(cases2, geojson=boroughs, locations='area_name', color='new_cases',
                           mapbox_style='stamen-toner',
                           color_continuous_scale='YlOrRd',
                           zoom=8, center = {"lat": 51.5074, "lon": -0.07},
                           animation_frame='week',
                           title='Weekly new COVID-19 cases during Spring 2020 lockdown',
                           labels={'new_cases':'New cases', 'area_name':'Borough', 'week':'Week'},
                           template='plotly_dark',
                           opacity=0.5)
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)'},margin={"r":0,"t":40,"l":0,"b":0})
fig.show()
fig.write_html("spring_dark.html")

### Socioeconomic data

In [ ]:
# Boroughs poverty rate, 0 = below average, 1 = average, 2 = above average
poverty_map = {'Barking and Dagenham': '1', 'Barnet': '1', 'Bexley': '2', 'Brent': '0', 'Bromley': '2', 'Camden': '1', 'City of London': '2',
               'Croydon': '1', 'Ealing': '1', 'Enfield': '1', 'Greenwich': '1', 'Hackney': '0', 'Hammersmith and Fulham': '1',
               'Haringey': '1', 'Harrow': '1', 'Havering': '2', 'Hillingdon': '1', 'Hounslow': '1', 'Islington': '0',
               'Kensington and Chelsea': '1', 'Kingston upon Thames': '2', 'Lambeth': '1', 'Lewisham': '1', 'Merton': '1', 'Newham': '0',
               'Redbridge': '1', 'Richmond upon Thames': '2', 'Southwark': '1', 'Sutton': '2', 'Tower Hamlets': '0', 'Waltham Forest': '1',
               'Wandsworth': '1', 'Westminster': '1'}

In [328]:
# change poverty map values 
for item in poverty_map.items():
    if item[1] == '0':
        poverty_map[item[0]] = 'Low'
    elif item[1] == '1':
        poverty_map[item[0]] = 'Average'
    elif item[1] == '2':
        poverty_map[item[0]] = 'High'

NameError: name 'poverty_map' is not defined

In [ ]:
# create dataframe of economic levels
df = pd.DataFrame(poverty_map.items(), columns=['locations', 'income'])
df.income = pd.Categorical(df.income, categories=['Low', 'Average', 'High'])
df = df.sort_values('income')

In [ ]:
# plot economic data as interactive map
with open('boroughs.json') as f:
    boroughs = json.load(f)

import plotly.express as px

fig = px.choropleth_mapbox(df, geojson=boroughs, locations='locations', color='income',
                           mapbox_style='stamen-toner',
                           color_discrete_sequence=px.colors.qualitative.Set1,
                           title='Relative income levels of London boroughs',
                           labels={'income':'Average incomes', 'locations': 'Borough'},
                           zoom=8, center = {"lat": 51.5074, "lon": -0.07},
                           template='plotly_dark',
                           opacity=0.5
                          )
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)'},margin={"r":0,"t":40,"l":0,"b":0})
fig.show()
fig.write_html("poverty_dark.html")

In [ ]:
# plot population data as interactive map
population = pd.read_csv('populations.csv',names=['locations','population','size',])

fig = px.choropleth_mapbox(population, geojson=boroughs, locations='locations', color='population',
                           mapbox_style='stamen-toner',
                           color_continuous_scale='YlOrRd',
                           title='Populations of London boroughs',
                           labels={'population':'Population', 'locations':'Borough'},
                           zoom=8, center = {"lat": 51.5074, "lon": -0.07},
                           template='plotly_dark',
                           opacity=0.5
                          )
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)'},margin={"r":0,"t":40,"l":0,"b":0})
fig.show()
fig.write_html('population_dark.html')

In [ ]:
# plot unemployment data as interactive map
unemployment = pd.read_csv('unemployment.csv')

fig = px.choropleth_mapbox(unemployment, geojson=boroughs, locations='locations', color='Unemployment',
                           mapbox_style='stamen-toner',
                           color_continuous_scale='YlOrRd',
                           title='Unemployment levels by borough',
                           labels={'Unemployment':'Percentage unemployed', 'locations': 'Borough'},
                           zoom=8, center = {"lat": 51.5074, "lon": -0.07},
                           template='plotly_dark',
                           opacity=0.5
                          )
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)'},margin={"r":0,"t":40,"l":0,"b":0})
fig.show()
fig.write_html('unemployment_dark.html')

In [ ]:
# plot labor data as interactive map
service = pd.read_csv('service.csv')

fig = px.choropleth_mapbox(service, geojson=boroughs, locations='Area', color='Total frontline workers',
                           mapbox_style='stamen-toner',
                           color_continuous_scale='YlOrRd',
                           title='Frontline workers by borough',
                           labels={'Area':'Borough'},
                           zoom=8, center = {"lat": 51.5074, "lon": -0.07},
                           template='plotly_dark',
                           opacity=0.5
                          )
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)'},margin={"r":0,"t":40,"l":0,"b":0})
fig.show()
fig.write_html('frontline_dark.html')